In [2]:
import pandas as pd
import numpy as np
import cv2
from pathlib import Path

In [15]:
root_dir = Path.cwd().parent
data_dir = root_dir / "data"

csv_file = data_dir / "raw/test_working.csv"

In [17]:
print(csv_file)

/home/dkenez/Projects/PFAS---final-project/data/raw/test_working.csv


In [18]:
object_columns = [
    "frame",
    "object_id",
    "object_type",
    "position_x",
    "position_y",
    "position_z",
    "last_seen",
    "bbox_cx",
    "bbox_cy",
    "bbox_w",
    "bbox_h",
]

In [20]:
#pandas read csv, sep = ",", columns = object_columns, header is first row

df = pd.read_csv(csv_file, sep = ",", names = object_columns, header = 0)
df.head()

,frame,object_id,object_type,position_x,position_y,position_z,last_seen,bbox_cx,bbox_cy,bbox_w,bbox_h
0,0,ed51f71c-75e3-4e33-85eb-191fd4932023,Car,8.634962,-0.458905,-19.733268,0,149.327255,280.016846,297.538971,178.187943
1,0,f0e1cb12-05e5-49b4-9aa0-edd4032d2ef5,Car,12.965250,4.762962,-71.912289,0,465.502411,182.963577,36.999084,24.450089
2,0,b460b033-048b-48d1-9ee1-9488e1e2b938,Pedestrian,-3.298049,0.492611,-14.617274,0,964.320557,218.467133,55.102417,149.353638
3,0,52e1221d-6cb0-4f79-a40b-b085c1db6c10,Pedestrian,0.925976,0.215627,-11.055321,0,584.129395,231.440948,64.872192,163.686493
4,0,b553d18a-6891-4d73-a688-7d68a3e017b9,Pedestrian,14.118380,0.406098,-31.200125,0,130.984802,227.547852,48.633926,143.959610


In [21]:
# get all unique object ids
object_ids = df["object_id"].unique()

# generate a random color for each object id
colors = np.random.randint(0, 255, size=(len(object_ids), 3), dtype=np.uint8)

# create a dictionary of object ids and colors
color_dict = dict(zip(object_ids, colors))

In [22]:
color_dict["b553d18a-6891-4d73-a688-7d68a3e017b9"]

array([ 87, 103,   6], dtype=uint8)

In [27]:
dp_left = data_dir / "raw"/"final_project_2023_rect" / "seq_03" / "image_02/data"

glob_left = sorted(dp_left.glob("*.png"))


In [28]:
print(glob_left[0])

/home/dkenez/Projects/PFAS---final-project/data/raw/final_project_2023_rect/seq_03/image_02/data/0000000000.png


In [31]:
data_dir / "processed" / "test" / "left"

PosixPath('/home/dkenez/Projects/PFAS---final-project/data/processed/test/left')

In [32]:
# for each image in the left image directory draw the bounding boxes with the colour of the object_id of the objects in the image and save the image
for frame, img_left_path in enumerate(glob_left):
    print(f"Processing frame {frame}: {img_left_path.name}")

    # read images
    img_left = cv2.imread(str(img_left_path))

    # get the rows of the dataframe for the current frame
    df_frame = df[df["frame"] == frame]

    # for each row in the dataframe draw the bounding box with the colour of the object_id
    for index, row in df_frame.iterrows():
        object_id = row["object_id"]
        bbox_cx = row["bbox_cx"]
        bbox_cy = row["bbox_cy"]
        bbox_w = row["bbox_w"]
        bbox_h = row["bbox_h"]

        # get the colour for the object_id
        color = tuple(color_dict[object_id])

        # draw the bounding box
        cv2.rectangle(
            img_left,
            (int(bbox_cx - bbox_w / 2), int(bbox_cy - bbox_h / 2)),
            (int(bbox_cx + bbox_w / 2), int(bbox_cy + bbox_h / 2)),
            (int(color[0]), int(color[1]), int(color[2])),
            2,
        )

    # save the image
    cv2.imwrite(
        str(data_dir / "processed" / "seqObjMatch" / img_left_path.name), img_left
    )


Processing frame 0: 0000000000.png
Processing frame 1: 0000000001.png
Processing frame 2: 0000000002.png
Processing frame 3: 0000000003.png
Processing frame 4: 0000000004.png
Processing frame 5: 0000000005.png
Processing frame 6: 0000000006.png
Processing frame 7: 0000000007.png
Processing frame 8: 0000000008.png
Processing frame 9: 0000000009.png
Processing frame 10: 0000000010.png
Processing frame 11: 0000000011.png
Processing frame 12: 0000000012.png
Processing frame 13: 0000000013.png
Processing frame 14: 0000000014.png
Processing frame 15: 0000000015.png
Processing frame 16: 0000000016.png
Processing frame 17: 0000000017.png
Processing frame 18: 0000000018.png
Processing frame 19: 0000000019.png
Processing frame 20: 0000000020.png
Processing frame 21: 0000000021.png
Processing frame 22: 0000000022.png
Processing frame 23: 0000000023.png
Processing frame 24: 0000000024.png
Processing frame 25: 0000000025.png
Processing frame 26: 0000000026.png
Processing frame 27: 0000000027.png
Pr